# Project Draft

#### Requirements
- ```Python 3.9.18``` (conda env)
- ```pip freeze > requirements.txt```
- ```conda env export > environment.yml```

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
#from pyspark.sql.functions import *
from pyspark.sql.functions import window, sum as _sum

In [ ]:
# load df from csv
df = spark.read.csv("streaming_data_dir/drivedata-small.csv", header=True, inferSchema=True)

In [ ]:
# show only rows where failure is not 0
df.where(df.failure != 0).show()


+-------------------+-------------+--------------------+-------+--------+------------------+-------------------------+---------------+-------------------+---------------------------+------------------+------------------------+-----------------+--------------------+---------------------+------------------------+--------------------------------+-------------------------+----------------------------------+--------------------+--------------------+----------------------------+-----------------------+----------------------------+----------------------------+------------------------+---------------------------+----------------------------+---------------------------+--------------------------+-------------------------+--------------------------+---------------+-----------------+---------------------+-----------------+----------------------+-----------------------+--------------------+
|               date|serial_number|               model|failure|vault_id|s1_read_error_rate|s2_throughput_pe

In [1]:



schema = StructType([
    StructField("date", TimestampType(), True),
    StructField("serial_number", StringType(), True),
    StructField("model", StringType(), True),
    StructField("failure", IntegerType(), True),
    StructField("vault_id", IntegerType(), True),
    StructField("s1_read_error_rate", IntegerType(), True),
    StructField("s2_throughput_performance", IntegerType(), True),
    StructField("s3_spin_up_time", IntegerType(), True),
    StructField("s4_start_stop_count", IntegerType(), True),
    StructField("s5_reallocated_sector_count", IntegerType(), True),
    StructField("s7_seek_error_rate", IntegerType(), True),
    StructField("s8_seek_time_performance", IntegerType(), True),
    StructField("s9_power_on_hours", IntegerType(), True),
    StructField("s10_spin_retry_count", IntegerType(), True),
    StructField("s12_power_cycle_count", IntegerType(), True),
    StructField("s173_wear_leveling_count", IntegerType(), True),
    StructField("s174_unexpected_power_loss_count", IntegerType(), True),
    StructField("s183_sata_downshift_count", IntegerType(), True),
    StructField("s187_reported_uncorrectable_errors", IntegerType(), True),
    StructField("s188_command_timeout", IntegerType(), True),
    StructField("s189_high_fly_writes", IntegerType(), True),
    StructField("s190_airflow_temperature_cel", IntegerType(), True),
    StructField("s191_g_sense_error_rate", IntegerType(), True),
    StructField("s192_power_off_retract_count", IntegerType(), True),
    StructField("s193_load_unload_cycle_count", IntegerType(), True),
    StructField("s194_temperature_celsius", IntegerType(), True),
    StructField("s195_hardware_ecc_recovered", IntegerType(), True),
    StructField("s196_reallocated_event_count", IntegerType(), True),
    StructField("s197_current_pending_sector", IntegerType(), True),
    StructField("s198_offline_uncorrectable", IntegerType(), True),
    StructField("s199_udma_crc_error_count", IntegerType(), True),
    StructField("s200_multi_zone_error_rate", IntegerType(), True),
    StructField("s220_disk_shift", IntegerType(), True),
    StructField("s222_loaded_hours", IntegerType(), True),
    StructField("s223_load_retry_count", IntegerType(), True),
    StructField("s226_load_in_time", IntegerType(), True),
    StructField("s240_head_flying_hours", IntegerType(), True),
    StructField("s241_total_lbas_written", IntegerType(), True),
    StructField("s242_total_lbas_read", IntegerType(), True)
])

spark = SparkSession.builder.appName("StreamingFromCSV").getOrCreate()


#read from csv file
#drivedata = spark.readStream.csv("streaming_data_dir/drivedata-small.csv", schema=schema, header=True)

# Read the CSV files as a data stream
streamingData = spark.readStream.schema(schema).option("maxFilesPerTrigger", 1).csv("streaming_data_dir")
# print true if both are true
print((streamingData.schema == schema)==( streamingData.isStreaming == True))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/26 15:44:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


True


### First Query

In [4]:

# Apply a window function to the streaming data
windowedData = streamingData \
    .withWatermark("date", "31 days") \
    .groupBy(
        streamingData.vault_id,
        window(streamingData.date, "30 days", "1 day"),
        streamingData.model
    ) \
    .agg(_sum("failure").alias("total_failures"))

# Select only the date and vault_id fields
selectedData = windowedData.select("window.start", "vault_id", "total_failures")
filteredData = selectedData.filter(selectedData.total_failures > 0)

# Write the windowed data stream out to a memory sink for testing
query = filteredData \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("filteredData") \
    .option("numRows", 50) \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/01/26 15:47:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/temporary-d0323577-d312-40a1-80a4-cf9f58ea260d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/01/26 15:47:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+--------+--------------+
|start              |vault_id|total_failures|
+-------------------+--------+--------------+
|2023-03-31 02:00:00|1149    |1             |
|2023-06-05 02:00:00|1042    |1             |
|2023-06-01 02:00:00|1097    |1             |
|2023-05-24 02:00:00|1042    |1             |
|2023-06-17 02:00:00|1406    |1             |
|2023-03-13 01:00:00|1124    |1             |
|2023-06-06 02:00:00|1406    |1             |
|2023-06-02 02:00:00|1406    |1             |
|2023-05-31 02:00:00|1042    |1             |
|2023-04-09 02:00:00|1149    |1             |
|2023-06-11 02:00:00|1042    |1             |
|2023-05-27 02:00:00|1406    |1             |
|2023-06-01 02:00:00|1406    |1             |
|2023-04-03 02:00:00|1149    |1             |
|2023-06-13 02:00:00|1042    |1             |
|2023-03-09 01:00:00|1124    |1             |
|2023-03-10 01:00:00|1124    

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/jonas/Documents/Uni/Erasmus/Cloud Computing for Big Data/Cloud-Computing-and-Big-Data-Applications/.conda/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/jonas/Documents/Uni/Erasmus/Cloud Computing for Big Data/Cloud-Computing-and-Big-Data-Applications/.conda/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/jonas/Documents/Uni/Erasmus/Cloud Computing for Big Data/Cloud-Computing-and-Big-Data-Applications/.conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
query.stop()